In [1]:
with open("txts/caravan.txt", "r") as f:
    text = f.read()

# Convert encoding from windows-1252 to utf-8
text = text.encode('windows-1252').decode('utf-8')
print(text[:1000])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xab in position 22: invalid start byte

In [30]:
# awk '/^[0-9][0-9]:[0-9][0-9]:[0-9][0-9]\.[0-9][0-9][0-9] --> /{getline; print}' surgery.vtt | sed 's/<[^>]*>//g' | uniq > surgery.txt

from langchain.text_splitter import TokenTextSplitter

with open("surgery.txt", "r") as f:
    text = f.read()

# Change \n to spaces
text = text.replace("\n", " ")

text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)

texts = text_splitter.split_text(text)

print(len(texts))
print(len(texts[0]))

31
955


In [37]:
import langchain
from langchain.llms import OpenAI
from langchain import PromptTemplate

OPENAI_API_KEY = "sk-xxx"
llm = OpenAI(model_name='text-davinci-003', openai_api_key=OPENAI_API_KEY, max_tokens=1200, model_kwargs={'temperature': 0.3})

name = "Полевая хирургия"

first_template_str = '''
Summarize the text below which is a beginning of subtitles for a youtube video called "{name}".
Pay attention and extract only to medical or historical information, ignore everything else.
IMPORTANT: Preserve maximum information and make the summary from 900 to 1000 words long.
START
{first}
{second}
END
SUMMARY:
'''

# Create prompt template
first_template = PromptTemplate(
    template = first_template_str,
    input_variables = ["first", "second", "name"],
)

rolling_template_str = '''
This is rolling summarization task of subtitles for a youtube video called "{name}. Analyze "NEW CHUNK" and incorporate the most important information from it with the "PREVIOUS SUMMARY" below, so we have a new summary.
In the new chunk pay attention and extract only to medical or historical information, ignore everything else.
IMPORTANT: Try preserving information but NEVER go above 970 words. Start removing information if necessary.
PREVIOUS SUMMARY
{summary}
NEW CHUNK
{chunk}
END
NEW SUMMARY:
'''


rolling_template = PromptTemplate(
    template = rolling_template_str ,
    input_variables = ["summary", "chunk", "name"],
)


In [38]:
summary = llm(first_template.format(first=texts[0], second=texts[1], name=name))
for i in range(2, len(texts)):
    summary = llm(rolling_template.format(summary=summary, chunk=texts[i], name=name))
    print(i, len(summary.split()))

print(len(summary.split()))
print(summary)

2 139
3 194
4 226
5 265
6 323
7 361
8 312
9 353
10 380
11 441
12 528
13 486
14 538
15 587
16 675
17 742
18 767
19 778
20 791
21 879
22 988
23 988
24 978
25 873
26 939
27 724
28 788
29 819
30 760
760
Alexander Povolotsky, a military historian and reconstructor, is joined by Mikhail Timin on the Tactic Media channel to discuss field surgery. They refer to the Polish company Wehrmacht which had around 10-15 thousand killed and 30-35 thousand wounded. In April 1940, a conference was held to discuss the experience of field surgery, and Sergey Yudin made a valuable contribution with his book "Notes on Military Field Surgery" which was signed on April 15, 1941. This book provided an unbiased view of German military medicine, which was praised by an English journalist. German statistics were discussed, with Sergey Smirnov noting that they were able to hide anything they wanted. The Germans concluded that there was nothing extraordinary in their field surgery, and they used local anesthesia and

In [39]:
refine_template_str = '''
Rewrite the text below to make it more fluent and coherent, but retain the meaning.
TEXT
{summary}
NEW REFINED TEXT:
'''

refine_template = PromptTemplate(
    template = refine_template_str ,
    input_variables = ["summary"],
)

refined_summary = llm(refine_template.format(summary=summary))
print(len(refined_summary.split()))
print(refined_summary)

760
Alexander Povolotsky, a military historian and reconstructor, and Mikhail Timin, joined forces on the Tactic Media channel to discuss field surgery. They referred to the Polish company Wehrmacht, which had around 10-15 thousand killed and 30-35 thousand wounded. In April 1940, a conference was held to discuss the experience of field surgery, and Sergey Yudin made a valuable contribution with his book "Notes on Military Field Surgery", which was signed on April 15, 1941. This book provided an unbiased view of German military medicine, which was praised by an English journalist. German statistics were discussed, with Sergey Smirnov noting that they were able to hide anything they wanted. The Germans concluded that there was nothing extraordinary in their field surgery, and they used local anesthesia and antiseptic techniques. Deep wound injections were still a controversial topic and required permission. The Germans had statistics on 35,000 wounded, but it took at least 6 months to p

In [40]:
jokes_template_str = '''
Find and extract jokes from the text below. For each joke write a one sentence summary. Format it as list with each line starting with an asterisk.
START
{text}
END
Jokes:
'''

jokes_template = PromptTemplate(
    template = jokes_template_str,
    input_variables = ["text"],
)

summaries = []
for text in texts:
    chunk_jokes = llm(jokes_template.format(text=text))
    print(len(chunk_jokes.split()))
    summaries.append(chunk_jokes)

all_jokes = "\n".join(summaries)
print(len(all_jokes.split()))
print(all_jokes)

42
66
75
59
38
85
36
52
27
141
53
65
34
18
45
48
53
21
88
12
82
87
63
79
87
43
47
19
39
97
34
1735
* "Ну понятно что польская компания Вермахта ее называли немножко бровивые": Joke about the name of the Polish company.
* "Потому что единственный американский солдат заболевший полиомиелитом умер весь понятно да": Joke about the 100% mortality rate of American soldiers with polio.
* The Germans held a conference in March 1940 to discuss military medicine, and Sergei Sergeevich Yudin gave everyone a priceless gift - a book of notes on military-field surgery, signed on April 15, 1941. 
* A British journalist was so impressed by the German military medicine that he didn't seem to mind that his breakdown of the results of German doctors' work was completely non-politicized.
* German statistics are made up of many numbers and graphs that are impossible to understand, so they are great for hiding anything.
* The Germans didn't say anything extraordinary after the results, which surprised the s

In [5]:
# Concat all summaries with newlines
summary = "\n".join(summaries)
print(len(summary.split()))


2522

Alexander Povolotsky, a military historian and re-enactor, is the guest on the Tactica Media channel. The topic of the day is a Polish company called Vermaht, which was nicknamed “the unshaven ones” due to their combat uniforms. It resulted in 10-15 thousand killed and 30-35 thousand wounded. This experience is ideal for military medicine, as it has great statistics. 

In April 1940, the Germans held a conference, where Sergey Yudin presented an invaluable gift - a book called “Notes on Military Field Surgery”, signed on April 15, 1941. It contained a lot of non-politicised information about the German military medicine and was very appreciated by an English journalist, who admired the German military medicine in particular. 

The Germans concluded that the primary surgical treatment was forbidden and deep antisepsis was required. They also used local anaesthetics, as those familiar with it in peace time. 

Overall, the Germans concluded nothing extraordinary, but the book was a 

In [8]:
final_template_str = '''
Summarize the text to become 3 times shorter but retain the meaning.

START
{text}
END

SUMMARY:
'''

final_template = PromptTemplate(
    template = final_template_str,
    input_variables = ["text"],
)

final_summary = llm(final_template.format(text=text))



In [36]:
import deepl
DEEPL_KEY = 'xxx'

# Translate text
translator = deepl.Translator(DEEPL_KEY)
result = translator.translate_text(summary, source_lang='EN', target_lang="RU" , preserve_formatting=True)

# Save translated text
with open("surgery_ru.txt", "w") as f:
    f.write(result.text)

In [ ]:
# https://colab.research.google.com/drive/10-GNzDd8-gNKiBBoxRqpcNgGhUF168Pk#scrollTo=nq0vKGFeW1KD
# https://gpt-index.readthedocs.io/en/latest/how_to/indices.html
# https://gpt-index.readthedocs.io/en/latest/guides/primer/index_guide.html
# Langchain agent: chat-conversational-react-description